In [1]:
import numpy as np
import plotly.graph_objects as go

# Domain
x = np.linspace(-5, 5, 1000)

# Helper heaviside
def heaviside(x):
    h = np.zeros_like(x)
    h[x > 0] = 1.0
    return h



In [4]:

# ReLU
relu = np.maximum(0, x)
drelu = heaviside(x)
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=x, y=relu, mode="lines", name="ReLU(x)"))
fig1.add_trace(go.Scatter(x=x, y=drelu, mode="lines", name="ReLU'(x)", line=dict(dash="dash")))
fig1.update_layout(title="ReLU and derivative", xaxis=dict(range=[-5,5]), yaxis=dict(range=[-10,10]))
fig1.show()

# Tanh
tanh = np.tanh(x)
dtanh = 1 - tanh**2
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=x, y=tanh, mode="lines", name="tanh(x)"))
fig2.add_trace(go.Scatter(x=x, y=dtanh, mode="lines", name="tanh'(x)", line=dict(dash="dash")))
fig2.update_layout(title="Tanh and derivative", xaxis=dict(range=[-5,5]), yaxis=dict(range=[-10,10]))
fig2.show()

# Sigmoid (and Softmax 2-class)
sig = 1/(1+np.exp(-x))
dsig = sig*(1-sig)
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=x, y=sig, mode="lines", name="Sigmoid(x)"))
fig3.add_trace(go.Scatter(x=x, y=dsig, mode="lines", name="Sigmoid'(x)", line=dict(dash="dash")))
fig3.update_layout(title="Sigmoid and derivative", xaxis=dict(range=[-5,5]), yaxis=dict(range=[-10,10]))
fig3.show()

# Leaky ReLU
alpha_leaky = 0.01
lrelu = np.where(x > 0, x, alpha_leaky*x)
dlrelu = np.where(x > 0, 1.0, alpha_leaky)
fig4 = go.Figure()
fig4.add_trace(go.Scatter(x=x, y=lrelu, mode="lines", name="Leaky ReLU(x)"))
fig4.add_trace(go.Scatter(x=x, y=dlrelu, mode="lines", name="Leaky ReLU'(x)", line=dict(dash="dash")))
fig4.update_layout(title="Leaky ReLU and derivative", xaxis=dict(range=[-5,5]), yaxis=dict(range=[-10,10]))
fig4.show()

# ELU
alpha_elu = 0.3
elu = np.where(x > 0, x, alpha_elu*(np.expm1(x)))
delu = np.where(x > 0, 1.0, alpha_elu*np.exp(x))
fig5 = go.Figure()
fig5.add_trace(go.Scatter(x=x, y=elu, mode="lines", name="ELU(x)"))
fig5.add_trace(go.Scatter(x=x, y=delu, mode="lines", name="ELU'(x)", line=dict(dash="dash")))
fig5.update_layout(title="ELU and derivative (α=0.3)", xaxis=dict(range=[-5,5]), yaxis=dict(range=[-10,10]))
fig5.show()

# Sinc
sinc = np.ones_like(x)
nz = x != 0
sinc[nz] = np.sin(x[nz])/x[nz]
dsinc = np.zeros_like(x)
dsinc[nz] = (x[nz]*np.cos(x[nz]) - np.sin(x[nz]))/(x[nz]**2)
fig6 = go.Figure()
fig6.add_trace(go.Scatter(x=x, y=sinc, mode="lines", name="sinc(x)"))
fig6.add_trace(go.Scatter(x=x, y=dsinc, mode="lines", name="sinc'(x)", line=dict(dash="dash")))
fig6.update_layout(title="Sinc and derivative", xaxis=dict(range=[-5,5]), yaxis=dict(range=[-10,10]))
fig6.show()

In [6]:
import numpy as np
import plotly.graph_objects as go

# Domain
x = np.linspace(-5, 5, 1000)

# Helper
def heaviside(x):
    h = np.zeros_like(x)
    h[x > 0] = 1.0
    return h

# Build data for each activation
alpha_leaky = 0.01
alpha_elu = 0.3

acts = {}

# ReLU
acts["ReLU"] = (
    np.maximum(0, x),
    heaviside(x),
    "ReLU(x)",
    "ReLU'(x)"
)

# Tanh
tanh = np.tanh(x)
acts["Tanh"] = (tanh, 1 - tanh**2, "tanh(x)", "tanh'(x)=1-tanh(x)^2")

# Softmax (2-class component = sigmoid)
sig = 1/(1+np.exp(-x))
acts["Softmax"] = (sig, sig*(1-sig), "σ1(x)", "dσ1/dx = σ1(1-σ1)")

# Sigmoid
acts["Sigmoid"] = (sig, sig*(1-sig), "σ(x)=1/(1+e^{-x})", "σ'(x)=σ(1-σ)")

# Leaky ReLU
lrelu = np.where(x > 0, x, alpha_leaky*x)
dlrelu = np.where(x > 0, 1.0, alpha_leaky)
acts[f"Leaky ReLU (α={alpha_leaky})"] = (lrelu, dlrelu, "LeakyReLU(x)", "derivative")

# ELU
elu = np.where(x > 0, x, alpha_elu*(np.expm1(x)))
delu = np.where(x > 0, 1.0, alpha_elu*np.exp(x))
acts[f"ELU (α={alpha_elu})"] = (elu, delu, "ELU(x)", "derivative")

# Sinc
sinc = np.ones_like(x)
nz = x != 0
sinc[nz] = np.sin(x[nz])/x[nz]
dsinc = np.zeros_like(x)
dsinc[nz] = (x[nz]*np.cos(x[nz]) - np.sin(x[nz]))/(x[nz]**2)
acts["Sinc"] = (sinc, dsinc, "sinc(x)=sin(x)/x", "sinc'(x)")

# Create a single figure with all traces, controlled by a dropdown
fig = go.Figure()

# Add all traces (2 per activation) but set all invisible except the first
names = list(acts.keys())
for i, name in enumerate(names):
    y, dy, y_lbl, dy_lbl = acts[name]
    visible = (i == 0)
    fig.add_trace(go.Scatter(x=x, y=y, mode="lines", name=y_lbl, visible=visible))
    fig.add_trace(go.Scatter(x=x, y=dy, mode="lines", name=dy_lbl, visible=visible, line=dict(dash="dash")))

# Build buttons to toggle visibility
buttons = []
for i, name in enumerate(names):
    vis = [False]* (2*len(names))
    vis[2*i] = True
    vis[2*i+1] = True
    buttons.append(dict(
        label=name,
        method="update",
        args=[{"visible": vis},
              {"title": f"{name} and derivative",
               "xaxis": {"range": [-5,5]},
               "yaxis": {"range": [-10,10]}}]
    ))

fig.update_layout(
    title=f"{names[0]} and derivative",
    xaxis_title="x",
    yaxis_title="Value",
    xaxis=dict(range=[-5,5]),
    yaxis=dict(range=[-10,10]),
    updatemenus=[dict(
        type="dropdown",
        direction="down",
        buttons=buttons,
        x=0.01, y=1.15, xanchor="left", yanchor="top",
        showactive=True
    )],
    legend=dict(orientation="h", yanchor="bottom", y=-0.2, xanchor="center", x=0.5),
    margin=dict(t=80)
)

fig.show()
